# Calculating Forest Loss in Protected Areas

In [22]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

In [23]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/muni_runapf')

# Import the Forest loss image
flossHansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# Import the Primary Tropical Forest cover image collection and filter it for Colombia
primaryForest_ideam = ee.Image("projects/ee-juamiji/assets/SouthAmerica_2001_primary").selfMask()
primaryForest_ideam_COL = primaryForest_ideam.clip(colombiaMpios)

# Clip the Forest loss image to Colombia's boundaries
flossHansen_COL = flossHansen.clip(colombiaMpios)

In [24]:
# Rename 'Municipi_5' to 'codmpio' for all features in the FeatureCollection
def rename_property(feature):
	return feature.set('codmpio', feature.get('Municipi_5'))

colombiaMpios = colombiaMpios.map(rename_property)
# Rename 'Municipi_5' to 'codmpio' for all features in the FeatureCollection
def rename_property(feature):
	return feature.set('codmpio', feature.get('Municipi_5'))

colombiaMpios = colombiaMpios.map(rename_property)

In [25]:
colombiaMpios.first().getInfo()


{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-75.63920502067865, 6.189718334872428],
    [-75.6391916141666, 6.189687102272576],
    [-75.63917820765333, 6.189655869691855],
    [-75.6391648899237, 6.189642512707483],
    [-75.63910691339325, 6.189557776132798],
    [-75.63917376841026, 6.189495362163453],
    [-75.6391648899237, 6.189490866758507],
    [-75.63916036189536, 6.189490899966914],
    [-75.63914260492007, 6.189481987894491],
    [-75.63904893683979, 6.189455245362634],
    [-75.639008806042, 6.189401705873237],
    [-75.639008806042, 6.189325883201634],
    [-75.639008806042, 6.18929021587215],
    [-75.639008806042, 6.189263445705022],
    [-75.63899539951197, 6.189187648981315],
    [-75.63898652101457, 6.189129692123202],
    [-75.63895082944963, 6.1891297362356],
    [-75.63892410516272, 6.189129711484527],
    [-75.63887056778917, 6.189120762424558],
    [-75.63867879196188, 6.189089595671147],
    [-75.63866547418677, 6.189098520980294],
 

In [26]:
# Get the property names (column names) of the shapefile
first_feature = colombiaMpios.first().getInfo()
column_names = list(first_feature['properties'].keys())
print("Column names:", column_names)

Column names: ['FID_Munici', 'FID_runapr', 'Municip_10', 'Municipi_1', 'Municipi_2', 'Municipi_3', 'Municipi_4', 'Municipi_5', 'Municipi_6', 'Municipi_7', 'Municipi_8', 'Municipi_9', 'Municipios', 'Shape_Area', 'Shape_Leng', 'codmpio']


In [27]:
# Visualization parameters for the map
viz_params = {
    'min':1,  # Minimum value for visualization
    'max': 1,  # Maximum value for visualization
    'palette': ['008000']  # Color gradient
}

# Initialize the map using geemap
Map = geemap.Map(center=[4.0, -72.0], zoom=5)  # Center roughly in Colombia with an appropriate zoom level
Map.addLayer(primaryForest_ideam_COL, viz_params, "Primary Tropical Forests")
Map

Map(center=[4.0, -72.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [28]:
# Create a mask where fcover is equal to 1.
primaryMask = primaryForest_ideam_COL.eq(1)

# Masking the floss imagery using the primary mask (cover equals 1)
flossHansen_COLmasked = flossHansen_COL.updateMask(primaryMask)

# Select the 'lossyear' band from the masked forest loss image
loss_year = flossHansen_COLmasked.select(['lossyear'])

## Code for all years from 2000 to 2023:

In [29]:
# Calculate the total loss area per municipality
def calculate_loss_area(feature, year):

    loss_year_mask = loss_year.eq(year)

    # Calculate the loss area for the current year in square meters
    loss_area_image = loss_year_mask.multiply(ee.Image.pixelArea())

    # Use reduceRegion to calculate the sum of the loss area within the feature's geometry
    loss_area = loss_area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('lossyear')  # Replace 'lossyear' with the correct property name if different
    
     # Dynamically name the loss area property
    loss_area_property_name = f'lossarea{year}'

    # Return a new feature with only the 'lossArea' and 'codmpio' properties
    return ee.Feature(None, {
        loss_area_property_name: loss_area,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

# Dictionary to store results for each year
year_results_dict = {}

for year in range(0, 24):
    # Use a lambda function to pass the current year into the map function
    current_year_results = colombiaMpios.map(lambda feature: calculate_loss_area(feature, year))
    year_results_dict[f'year_results{year}'] = current_year_results
    # Optionally print the first feature's info to check
    print(year_results_dict[f'year_results{year}'].first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'codmpio': 5360, 'lossarea0': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'codmpio': 5360, 'lossarea1': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'codmpio': 5360, 'lossarea2': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'codmpio': 5360, 'lossarea3': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'codmpio': 5360, 'lossarea4': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'codmpio': 5360, 'lossarea5': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'codmpio': 5360, 'lossarea6': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'codmpio': 5360, 'lossarea7': 0}}
{'type': 'Feature', 'geometry': None, 'id': '0000000000000000020

In [30]:
# Define a function to export FeatureCollections to Google Drive
def export_to_drive(feature_collection, year, description_prefix="ForestLossIllegalData", folder_name="EarthEngineExports"):
    export_task = ee.batch.Export.table.toDrive(
        collection=feature_collection,
        description=f"{description_prefix}_Year{year}",   # description is task name only
        folder=folder_name,  
        fileNamePrefix=f"ForestLoss_Illegal_Year{year}",  # file name, unique per year
        fileFormat="CSV"
    )
    # Start the export task
    export_task.start()
    print(f"Exporting {description_prefix}_Year{year} to Google Drive...")

# Loop through each item in the dictionary and initiate an export
for year, feature_collection in year_results_dict.items():
    # Extract the year from the key (which is a string like 'year_results11')
    extracted_year = int(year.replace('year_results', ''))
    export_to_drive(feature_collection, extracted_year)

Exporting ForestLossIllegalData_Year0 to Google Drive...
Exporting ForestLossIllegalData_Year1 to Google Drive...
Exporting ForestLossIllegalData_Year2 to Google Drive...
Exporting ForestLossIllegalData_Year3 to Google Drive...
Exporting ForestLossIllegalData_Year4 to Google Drive...
Exporting ForestLossIllegalData_Year5 to Google Drive...
Exporting ForestLossIllegalData_Year6 to Google Drive...
Exporting ForestLossIllegalData_Year7 to Google Drive...
Exporting ForestLossIllegalData_Year8 to Google Drive...
Exporting ForestLossIllegalData_Year9 to Google Drive...
Exporting ForestLossIllegalData_Year10 to Google Drive...
Exporting ForestLossIllegalData_Year11 to Google Drive...
Exporting ForestLossIllegalData_Year12 to Google Drive...
Exporting ForestLossIllegalData_Year13 to Google Drive...
Exporting ForestLossIllegalData_Year14 to Google Drive...
Exporting ForestLossIllegalData_Year15 to Google Drive...
Exporting ForestLossIllegalData_Year16 to Google Drive...
Exporting ForestLossIlle